In [80]:
from horton import *
import numpy as np
from numpy.linalg import norm
from numpy import dot,cross
Methane=IOData.from_file('opt_uhf_631g.xyz')
#print Methane.numbers
#Methane.coordinates*=angstrom

In [86]:
def uhf (mol,obasis):
    lf=DenseLinalgFactory(obasis.nbasis)
    #orbital integrals
    olp=obasis.compute_overlap(lf)
    kin=obasis.compute_kinetic(lf)
    na=obasis.compute_nuclear_attraction(mol.coordinates,mol.pseudo_numbers,lf)
    er=obasis.compute_electron_repulsion(lf)
    
    exp_alpha=lf.create_expansion()
    exp_beta=lf.create_expansion()

    guess_core_hamiltonian(olp, kin, na, exp_alpha,exp_beta)
    occ_model=AufbauOccModel(5,5)
    occ_model.assign(exp_alpha, exp_beta)
    
    external = {'nn': compute_nucnuc(mol.coordinates, mol.pseudo_numbers)}
    terms = [
        UTwoIndexTerm(kin, 'kin'),
        UDirectTerm(er, 'hartree'),
        UExchangeTerm(er, 'x_hf'),
        UTwoIndexTerm(na, 'ne'),
    ]
    ham = UEffHam(terms, external)
    dm_alpha = exp_alpha.to_dm()
    dm_beta = exp_beta.to_dm()
    # - SCF solver
    scf_solver = EDIIS2SCFSolver(1e-10,maxiter=200)
    scf_solver(ham, lf, olp, occ_model, dm_alpha, dm_beta)
    
    grid=BeckeMolGrid(mol.coordinates,mol.numbers,mol.numbers, random_rotate=False,agspec='insane')
    rho_alpha = obasis.compute_grid_density_dm(dm_alpha, grid.points)
    rho_beta = obasis.compute_grid_density_dm(dm_beta, grid.points)
    rho=rho_alpha+rho_beta
    
    return ham.cache['energy'],rho 

In [87]:
basis_set='6-31G'
ch4_basis=get_gobasis(Methane.coordinates,Methane.numbers,basis_set)
N_basis=get_gobasis(Methane.coordinates[0:1],np.array([7]),basis_set)
alc_basis=GOBasis.concatenate(ch4_basis,N_basis)

In [88]:
uhf(Methane,alc_basis)

(-40.18179111538183,
 array([1.18254112e+02, 1.18254112e+02, 1.18254112e+02, ...,
        1.90724716e-59, 2.53810766e-67, 3.07801270e-51]))

In [89]:
gridc=BeckeMolGrid(Methane.coordinates,Methane.numbers,Methane.numbers, random_rotate=False,agspec='insane')

In [90]:
e0,rho0=uhf(Methane,alc_basis)

In [91]:
h=0.05
Methane.pseudo_numbers[0]+=h
Methane.pseudo_numbers[4]-=h
e1,rho1=uhf(Methane,alc_basis)

In [92]:
Methane.pseudo_numbers[0]-=2*h
Methane.pseudo_numbers[4]+=2*h
e2,rho2=uhf(Methane,alc_basis)

In [93]:
Methane.pseudo_numbers

array([5.95, 1.  , 1.  , 1.  , 1.05])

In [94]:
Ammonia=IOData()
Ammonia.coordinates=Methane.coordinates[:-1]
Ammonia.numbers=numbers=np.array([7,1,1,1])
Ammonia.pseudo_numbers=np.array([7.,1.,1.,1.])
nh3_basis=get_gobasis(Ammonia.coordinates,Ammonia.numbers,basis_set)
C_basis=get_gobasis(Methane.coordinates[0:1],np.array([6]),basis_set)
nh3c_basis=GOBasis.concatenate(nh3_basis,C_basis)

In [101]:
DVNN=compute_nucnuc(Ammonia.coordinates,Ammonia.pseudo_numbers)\
-compute_nucnuc(Methane.coordinates, Methane.pseudo_numbers)
DVNN

-2.458099230232188

In [96]:
e_nh3,rho_nh3 = uhf(Ammonia,nh3_basis)
e_nh3c,rho_nh3c = uhf(Ammonia,nh3c_basis)
e_nh3,e_nh3c

(-56.148064068857224, -56.15385511303025)

In [97]:
# alchemy second order in energy- first in density 
coordinates=Methane.coordinates
V1=((gridc.points[:, 0]-coordinates[0][0])**2+(gridc.points[:, 1]-coordinates[0][1])**2+\
    (gridc.points[:, 2]-coordinates[0][2])**2)**-0.5
V2=((gridc.points[:, 0]-coordinates[4][0])**2+(gridc.points[:, 1]-coordinates[4][1])**2+\
    (gridc.points[:, 2]-coordinates[4][2])**2)**-0.5
DV=V2-V1

In [102]:
E_est=e0+gridc.integrate(rho0,DV)+0.5*gridc.integrate(rho1-rho2,DV)/2./h
E_est,E_est+DVNN

(-53.91419787274343, -56.372297102975615)

In [103]:
E_est=e0+gridc.integrate(rho0,DV)+0.5*gridc.integrate(rho1-rho2,DV)/2/h\
+1/6.*gridc.integrate(rho1-2*rho0+rho2,DV)/h**2
E_est,E_est+DVNN

(-53.79752044795264, -56.25561967818483)

.1 Angstrom of error  -56.25561967818483 vs -56.15385511303025 